In [1]:
import numpy as np # linear algebra
import pandas as pd
import pyarrow.parquet as pq
import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense
import numpy as np
import os
import h5py
#from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.decomposition import PCA

['weights-1', 'traindata', 'dsadasd']


In [4]:
def find(SNR):
    #H_R.shape = (512,56,924,5)
    temp = np.sum(SNR**2, axis=1)
    #print(temp.shape)
    idx  = np.argmax(temp, axis=1)
    return idx
    
def preprocess(H_Re, idx):
    temp = np.zeros(H_Re.shape[:-1])
    for i in range(H_Re.shape[0]):
        temp[i] = H_Re[i,:,:,idx[i]]
    return temp

def preprocess2(H_Re, idx):
    temp = np.zeros(H_Re.shape[:-1])
    for i in range(H_Re.shape[0]):
        temp[i] = H_Re[i,:,idx[i]]
    return temp
      
        

In [6]:
def get_data(data_file):
    f = h5py.File(data_file, 'r')
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR = f['SNR'][:] #shape (sample size, 56, 5)
    Pos = f['Pos'][:] #shape(sample size, 3)
    f.close()
    return H_Re, H_Im, SNR, Pos

CTW_labelled = "../input/traindata"
data_file = CTW_labelled+"/file_"+str(1)+".hdf5"
H_Re, H_Im, SNR, Pos = get_data(data_file)

#print(H_Re[1,1,1,:])

idx = find(SNR)

H_Re = preprocess(H_Re, idx)
H_Im = preprocess(H_Im, idx)
SNR = preprocess2(SNR, idx)
#print(SNR.shape)



[[  15.41993422   20.02795521 -100.         ...   20.20389977
    20.98391009   13.45937879]
 [  14.87486152   15.71887731   12.85714942 ...   20.89376565
    20.68049551   23.21826983]
 [-100.           10.50935505 -100.         ...   22.51211621
    21.43964909   21.95467171]
 ...
 [-100.         -100.         -100.         ... -100.
  -100.         -100.        ]
 [  17.78958193   16.50816507 -100.         ...   22.56948133
    18.74475369   21.48719758]
 [-100.         -100.         -100.         ... -100.
  -100.         -100.        ]]


In [7]:
for i in range(2,3):
    temp = CTW_labelled+"/file_"+str(i)+".hdf5"
    tH_Re, tH_Im, tSNR, tPos = get_data(temp)
    idx = find(tSNR)
    tH_Re = preprocess(tH_Re, idx)
    tH_Im = preprocess(tH_Im, idx)
    tSNR = preprocess2(tSNR, idx)
    H_Re, H_Im, Pos, SNR  = np.concatenate((H_Re, tH_Re)), np.concatenate((H_Im, tH_Im)), np.concatenate((Pos, tPos)), np.concatenate((SNR, tSNR))

In [8]:
#print(H_Re[:,1,1])

samples = H_Re.shape[0]
H_Re = H_Re.reshape((samples,-1))
print(H_Re.shape)
H_Im = H_Im.reshape((samples,-1))
#SNR = SNR.reshape((samples,-1))
#Pos = Pos.reshape((samples,-1))

data = np.concatenate((H_Re, H_Im, SNR), axis=1)
print(data.shape)


(1024, 51744)
(1024, 103544)


In [ ]:
pca = PCA(n_components=24)
o = pca.fit_transform(data[:1000])
y = pca.explained_variance_ratio_
#x = np.linspace(1,data[:1000].shape[1], data[:1000].shape[1])
#plt.plot(x,y)
print(o.shape)

In [ ]:
print(data[:int(data.shape[0]/5)].shape)

In [ ]:
'''

model = Sequential()
model.add(Dense(compression_1, activation = 'relu', input_shape=(517718,), name = 'compress'))
model.add(Dense(517718, activation='linear'))
model.compile(loss= 'mean_squared_error', optimizer = 'adam')
#model.loadweights('../input/weights_1/saved_model.pb')
#print('model summary')
'''





In [9]:
def create_model():
    model = tf.keras.Sequential([ tf.keras.layers.Dense(20, activation='relu', input_shape=(103544,), name='compress'),
                           tf.keras.layers.Dense(103544, activation='linear')])
    model.compile(loss= 'mean_squared_error', optimizer = 'adam') 
    return model
print(data.shape)

(1024, 103544)


In [10]:
#compression_1 = 20
model = create_model()
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 3, verbose=1)
checkpointer = ModelCheckpoint('Best', verbose=1, save_best_only=True)
results = model.fit(data[:1000], data[:1000], validation_split=0.1, batch_size = 10, epochs = 100, callbacks=[earlystopper, checkpointer])
model.save('mymodel.h5')

Using TensorFlow backend.


Train on 900 samples, validate on 100 samples
Epoch 1/100
890/900 [============================>.] - ETA: 0s - loss: 1.7168
Epoch 00001: val_loss improved from inf to 0.69042, saving model to Best
900/900 [==============================] - 4s 5ms/sample - loss: 1.7094 - val_loss: 0.6904
Epoch 2/100
890/900 [============================>.] - ETA: 0s - loss: 0.6393
Epoch 00002: val_loss improved from 0.69042 to 0.56697, saving model to Best
900/900 [==============================] - 3s 3ms/sample - loss: 0.6381 - val_loss: 0.5670
Epoch 3/100
870/900 [============================>.] - ETA: 0s - loss: 0.5876
Epoch 00003: val_loss improved from 0.56697 to 0.55504, saving model to Best
900/900 [==============================] - 3s 4ms/sample - loss: 0.5858 - val_loss: 0.5550
Epoch 4/100
880/900 [============================>.] - ETA: 0s - loss: 0.5688
Epoch 00004: val_loss improved from 0.55504 to 0.53770, saving model to Best
900/900 [==============================] - 3s 3ms/sample - loss: 

In [12]:
print(data[1002])
print(np.sum(((model.predict(data[1001:])[0])-data[1001])**2)/103544)

[   0.    0.    0. ... -100. -100. -100.]
0.01730572994244381


In [ ]:
results = model.fit(data[int(data.shape[0]/5):2*int(data.shape[0]/5)], data[int(data.shape[0]/5):2*int(data.shape[0]/5)], validation_split=0.1, batch_size = 10, epochs = 100, callbacks=[earlystopper, checkpointer])

In [ ]:
results = model.fit(data[2*int(data.shape[0]/5):3*int(data.shape[0]/5)], data[2*int(data.shape[0]/5):3*int(data.shape[0]/5)], validation_split=0.1, batch_size = 10, epochs = 100, callbacks=[earlystopper, checkpointer])

In [ ]:
c_model = create_model()
#print(os.listdir("../output"))

c_model = load_model('../input/dsadasd/Best')

In [ ]:
X_decompressed = c_model.predict(data[0:6])
print(np.sum((X_decompressed - data[0:6])**2)/517784*6)